<a href="https://colab.research.google.com/github/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/blob/main/Copie_de_New_Notebook_Blancpain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DMML TEAM BLANCPAIN

##Import some librairies

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
sns.set_style("white")
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.multiclass import unique_labels
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import spacy
from gensim.models import Word2Vec

%matplotlib inline
sns.set_style("dark")



##Data Importation##

In [4]:
df_train = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/training_data.csv')

df_test = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/test_data.csv')

df_sample = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/sample_submission.csv')

##Data visualization##

###Training set

In [5]:
df_train.head()

,id,keyword,location,text,target
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1


In [6]:
print("There are {0} rows and {1} columns in the train dataset.".format(df_train.shape[0],df_train.shape[1]))

There are 6471 rows and 5 columns in the train dataset.


###Test Set

In [7]:
df_test.head()

,id,keyword,location,text
0,9972,tsunami,NaN,Crptotech tsunami and banks.\n http://t.co/KHz...
1,9865,traumatised,"Portsmouth, UK",I'm that traumatised that I can't even spell p...
2,1937,burning%20buildings,NaN,@foxnewsvideo @AIIAmericanGirI @ANHQDC So ... ...
3,3560,desolate,NaN,Me watching Law &amp; Order (IB: @sauldale305)...
4,2731,crushed,bahstun/porta reeko,Papi absolutely crushed that ball


In [8]:
print("There are {0} rows and {1} columns in the test dataset.".format(df_test.shape[0],df_test.shape[1]))

There are 1142 rows and 4 columns in the test dataset.


In [9]:
df_sample.head()

,target
0,0
1,0
2,0
3,0
4,0


###Drop Null values

In [10]:
df_train.fillna("Unknown", inplace = True)
df_test.fillna("Unknown", inplace = True)

###Base Rate 

In [11]:
base_rate = max(len(df_train[df_train["target"] == 0]) / len(df_train), len(df_train[df_train["target"] == 1]) / len(df_train))
print("The base rate for this problem is :", base_rate)

The base rate for this problem is : 0.5719363313243703


##Processing

In [12]:
df_train["keyword_text"] = df_train['keyword'] + ' ' + df_train['text'] 
df_train["keyword_text"] = df_train["keyword_text"].astype(str)
df_test["keyword_text"] = df_test['keyword'] + ' ' + df_test['text'] 
df_test["keyword_text"] = df_test["keyword_text"].astype(str)

In [13]:
df_train

,id,keyword,location,text,target,keyword_text
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0,destroyed Black Eye 9: A space battle occurred...
1,853,bioterror,Unknown,#world FedEx no longer to transport bioterror ...,0,bioterror #world FedEx no longer to transport ...
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1,windstorm Reality Training: Train falls off el...
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1,hazardous #Taiwan Grace: expect that large roc...
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1,hostage New ISIS Video: ISIS Threatens to Behe...
...,...,...,...,...,...,...
6466,4377,earthquake,ARGENTINA,#Earthquake #Sismo M 1.9 - 15km E of Anchorage...,1,earthquake #Earthquake #Sismo M 1.9 - 15km E o...
6467,3408,derail,Unknown,@EmiiliexIrwin Totally agree.She is 23 and kno...,0,derail @EmiiliexIrwin Totally agree.She is 23 ...
6468,9794,trapped,Unknown,Hollywood Movie About Trapped Miners Released ...,1,trapped Hollywood Movie About Trapped Miners R...
6469,10344,weapons,Beirut/Toronto,Friendly reminder that the only country to eve...,1,weapons Friendly reminder that the only countr...


# Create validation set to test our best model

In [12]:
df_train_eval, df_validation = train_test_split(df_train, test_size=0.2, random_state = 50)

In [13]:
y_train_eval = df_train_eval[["target"]]
y_validation = df_validation[["target"]]

###With "Keyword" feature

In [14]:
X1_train_eval = df_train_eval['keyword']
X1_validation = df_validation['keyword']

In [15]:
# Load English language model
sp = spacy.load('en_core_web_sm')

# Create a list of punctuation marks
punctuations = string.punctuation

# Create a list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def tokenize_function(text):
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = sp(text)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    ## Remove stop words and punctuation
    #mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    return mytokens

In [16]:
tokenize_function("j'aime le machine learning")

["j'aime", 'le', 'machine', 'learning']

In [17]:
tfidf_feature_vector = TfidfVectorizer(tokenizer=tokenize_function)

In [18]:
l_reg_CV = LogisticRegressionCV(solver='lbfgs', cv=2, max_iter=1000, random_state=50)

In [19]:
pipe = Pipeline([('vectorizer', tfidf_feature_vector), 
                 ('classifier', l_reg_CV)])

In [20]:
pipe.fit(X1_train_eval, y_train_eval)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                                 tokenizer=<function tokenize_function at 0x7f3306468950>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LogisticRegressionCV(Cs=10, class_weight=None, cv=2,
                                      dual=F

In [21]:
train_accuracy_score = accuracy_score(y_train_eval, pipe.predict(X1_train_eval))
print("The training accuracy is", train_accuracy_score)

The training accuracy is 0.7436244204018547


In [22]:
score_test = accuracy_score(y_validation, pipe.predict(X1_validation))
print("The test accuracy is", score_test)

The test accuracy is 0.6988416988416989


In [35]:
import gensim.downloader

def eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword', feature_engineering='tfidf'):
  ''' Eval pipeline '''

  train = df_train.copy()
  test = df_test.copy()

  model = LogisticRegressionCV(solver='lbfgs', cv=2, max_iter=1000, random_state=50)
  
  def document_vector(doc):
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    tokens = tokenize_function(doc)
    doc = [word for word in tokens if word in feature_vector.wv.vocab]
    return np.mean(feature_vector[doc], axis=0)

  # Define embeddings and model
  if feature_engineering == 'tfidf':
    feature_vector = TfidfVectorizer(tokenizer=tokenize_function)
    model = Pipeline([('vectorizer', feature_vector), 
                   ('classifier', model)])
  
  elif feature_engineering == 'w2v':
    # Create corpus for w2v
    sentences = []
    for text in [list(df_train[target_col]), list(df_test[target_col])]:
      for string in text:
        sentences.append(tokenize_function(string))

    # Create w2v embeddings
    feature_vector = gensim.models.Word2Vec(sentences, size=100, window=10, min_count=2, iter=20)
    train[target_col] = train[target_col].apply(document_vector)
    test[target_col] = test[target_col].apply(document_vector)
  
  # Split dataset
  df_train_eval, df_validation = train_test_split(train, test_size=0.2, random_state = 50)

  y_train_eval = df_train_eval[["target"]]
  y_validation = df_validation[["target"]]

  X1_train_eval = list(df_train_eval[target_col])
  X1_validation = list(df_validation[target_col])

  # Fit data
  model.fit(X1_train_eval, y_train_eval)

  # Eval data
  train_accuracy_score = accuracy_score(y_train_eval, model.predict(X1_train_eval))
  print("The training accuracy is", train_accuracy_score)

  score_test = accuracy_score(y_validation, model.predict(X1_validation))
  print("The test accuracy is", score_test)

  y_train = train[["target"]]
  X1_train = list(train[target_col])
  X1_test = list(test[target_col])

  # Create submission file
  model_w2v.fit(X1_train, y_train)
  submission = pd.DataFrame(model.predict(X1_test), columns=["target"])
  submission.to_csv(f'submission_{target_col}_{feature_engineering}_model_LogisticRegressionCV2_test1.csv', index=False)

In [37]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword', feature_engineering='w2v')
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword', feature_engineering='tfidf')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The training accuracy is 0.6895285935085008
The test accuracy is 0.6486486486486487


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The training accuracy is 0.7436244204018547
The test accuracy is 0.6988416988416989


ValueError: ignored

# Submission with keywords

In [34]:
y_train = df_train[["target"]]
X1_train = list(df_train['keyword'])
X1_test = list(df_test['keyword'])

model_w2v.fit(X1_train, y_train)
submission = pipe.predict(X1_test)

ValueError: ignored

In [ ]:
submission = pd.DataFrame(submission, columns=["target"])
submission.to_csv('submission_keywords_w2v_model_LogisticRegressionCV2_test1.csv', index=False)

In [ ]:
y_train = df_train[["target"]]
X1_train = list(df_train['keyword'])
X1_test = list(df_test['keyword'])

model_tfidf.fit(X1_train, y_train)
submission = pipe.predict(X1_test)

In [ ]:
submission = pd.DataFrame(submission, columns=["target"])
submission.to_csv('submission_keywords_tfidf_model_LogisticRegressionCV2_test1.csv', index=False)

###With "Text" Feature

In [23]:
X2_train_train = df_train_train["text"]
X2_train_test = df_train_test["text"]

NameError: ignored

In [ ]:
sp = spacy.load('en_core_web_sm')

def tokenize_function(tweet):
    tokens = sp(tweet)
    tokens = [ i.lower_ for i in tokens ]
    return tokens

In [ ]:
tfidf_feature_vector = TfidfVectorizer(tokenizer=tokenize_function)

In [ ]:
classifierCV = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=10000, random_state=50)

In [ ]:
pipe = Pipeline([('vectorizer', tfidf_feature_vector),
                 ('classifier', classifierCV)])

In [ ]:
pipe.fit(X2_train_train, y_train_train)

In [ ]:
y_pred = pipe.predict(X2_train_test)

In [ ]:
score_test = accuracy_score(y_train_test, y_pred)
print("The test accuracy is", score_test)

In [ ]:
train_accuracy_score = accuracy_score(y_train_train, pipe.predict(X2_train_train))
print("The training accuracy is", train_accuracy_score)

In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='text', feature_engineering='w2v')
eval_pipeline(df_train, df_test, tokenize_function, target_col='text', feature_engineering='tfidf')

###With "Text" and "Keyword" features

In [ ]:
X3_train_train = df_train_train["keyword_text"]
X3_train_test = df_train_test["keyword_text"]

In [ ]:
sp = spacy.load('en_core_web_sm')

def tokenize_function(tweet):
    tokens = sp(tweet)
    tokens = [ i.lower_ for i in tokens ]
    return tokens

In [ ]:
tfidf_feature_vector = TfidfVectorizer(tokenizer=tokenize_function)
classifierCV = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=10000, random_state=50)

In [ ]:
pipe = Pipeline([('vectorizer', tfidf_feature_vector),
                 ('classifier', classifierCV)])

In [ ]:
pipe.fit(X3_train_train, y_train_train)

In [ ]:
y_pred = pipe.predict(X3_train_test)

In [ ]:
score_test = accuracy_score(y_train_test, y_pred)
print("The test accuracy is", score_test)

In [ ]:
train_accuracy_score = accuracy_score(y_train_train, pipe.predict(X3_train_train))
print("The training accuracy is", train_accuracy_score)

In [ ]:
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_text', feature_engineering='w2v')
eval_pipeline(df_train, df_test, tokenize_function, target_col='keyword_text', feature_engineering='tfidf')